In [ ]:
!pip install transformers --quiet
!pip install allennlp==1.0.0 allennlp-models==1.0.0 --quiet
#!pip install --user gradio

from urllib import request
from bs4 import BeautifulSoup as bs
import re
import nltk
import heapq
from transformers import pipeline
#import gradio as gr

nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
url = 'https://en.wikipedia.org/wiki/Olympic_Games'
TotalContent = ""

htmlDoc = request.urlopen(url)

soupObj = bs(htmlDoc, 'html.parser')
paragraphContents = soupObj.findAll('p')

for content in paragraphContents:

  TotalContent += content.text



total_cleaned_content = re.sub(r'\[[0-9]*\]', ' ', TotalContent)
total_cleaned_content = re.sub(r'\s+', ' ', total_cleaned_content)

sentence_tokens = nltk.sent_tokenize(total_cleaned_content)

total_cleaned_content = re.sub(r'[^a-zA-Z]', ' ', total_cleaned_content)
total_cleaned_content = re.sub(r'\s+', ' ', total_cleaned_content)

print(sentence_tokens)

In [ ]:
words_tokens = nltk.word_tokenize(total_cleaned_content)

stopwords = nltk.corpus.stopwords.words('english')

word_frequencies = {}

for word in words_tokens:

  if word not in stopwords:

    if word not in word_frequencies.keys():

      word_frequencies[word] = 1
    else:

      word_frequencies[word] += 1


print(words_tokens)


In [ ]:
maximum_frequency = max(word_frequencies.values())

for word in word_frequencies.keys():
  word_frequencies[word] = (word_frequencies[word]/maximum_frequency)

print(word_frequencies)


In [5]:
sentence_scores = {}

for sentence in sentence_tokens:

  for word in nltk.word_tokenize(sentence.lower()):

    if word in word_frequencies.keys():

      if (len(sentence.split(' '))) < 30:

        if sentence not in sentence_scores.keys():
          sentence_scores[sentence] = word_frequencies[word]
        else:
          sentence_scores[sentence] += word_frequencies[word]


points = heapq.nlargest(20, sentence_scores, key=sentence_scores.get)


In [6]:
total_text = ""

for i in points:

    total_text += i
    

In [7]:
summarizer = pipeline("summarization") # model="t5-small"

In [8]:
summary = summarizer(total_text, max_length=128, min_length=64, do_sample=False)

In [9]:
summary[0]["summary_text"]

'The Olympic Games are normally held every four years, alternating between the Summer and Winter Olympics every two years in the four-year period. Current Olympic sports, like badminton, basketball, and volleyball, first appeared on the programme as demonstration sports, and were later promoted to full Olympic sports. The Summer Olympics programme includes 26 sports, while the Winter Olympics programme features 15 sports.'

In [2]:
from allennlp.predictors.predictor import Predictor
## Old Model commented
# predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/bidaf-model-2017.09.15-charpad.tar.gz")
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/bidaf-model-2020.03.19.tar.gz")



In [11]:
result = predictor.predict(
  passage=TotalContent,
  question="how many nations participating in the olympics?"
)
result['best_span_str']

'more than 200'

In [13]:
print("Bullet Points: ")

string = """"""

for i in range(5):
    
   print("\n - " + points[i])

   string += "\n - " + points[i]

print("\n Summary: " + str(summary[0]["summary_text"]))
print(string)

Bullet Points: 

 - Current Olympic sports, like badminton, basketball, and volleyball, first appeared on the programme as demonstration sports, and were later promoted to full Olympic sports.

 - Nations then enter the stadium alphabetically according to the host country's chosen language, with the host country's athletes being the last to enter.

 - Beijing is due to host the 2022 Winter Olympics, which will make it the first city to host both the Summer and Winter Games.

 - Olympic sports are governed by international sports federations (IFs) recognised by the IOC as the global supervisors of those sports.

 - The 2020 Summer Olympics is due to be held in Tokyo, which will be the first Asian city to host the Olympics for the second time.

 Summary: The Olympic Games are normally held every four years, alternating between the Summer and Winter Olympics every two years in the four-year period. Current Olympic sports, like badminton, basketball, and volleyball, first appeared on the p

# **Results**


---


Bullet Points

* Current Olympic sports, like badminton, basketball, and volleyball, first appeared on the programme as demonstration sports, and were later promoted to full Olympic sports.

- Nations then enter the stadium alphabetically according to the host country's chosen language, with the host country's athletes being the last to enter.

- Beijing is due to host the 2022 Winter Olympics, which will make it the first city to host both the Summer and Winter Games.

- Olympic sports are governed by international sports federations (IFs) recognised by the IOC as the global supervisors of those sports.

- The 2020 Summer Olympics is due to be held in Tokyo, which will be the first Asian city to host the Olympics for the second time.

- With the addition of women's boxing to the programme in the 2012 Summer Olympics, however, female athletes were able to compete in all the same sports as men.

- At the 2012 Summer Olympics, every participating nation included female athletes for the first time in Olympic history.

- Since there was no agreement in the promotion of two other sports, the 2012 programme featured just 26 sports.

- The first Summer Youth Games were held in Singapore from 14 to 26 August 2010, while the inaugural Winter Games were hosted in Innsbruck, Austria, two years later.

- The Olympic Games are normally held every four years, alternating between the Summer and Winter Olympics every two years in the four-year period.



---


**Generated Summary**

Olympic sports are governed by international sports federations (IFs) recognised by the IOC as the global supervisors of those sports. Current Olympic sports, like badminton, basketball, and volleyball, first appeared on the programme as demonstration sports, and were later promoted to full Olympic sports. At the 2012 Summer Olympics, every participating nation included female athletes for the first time in Olympic history. Since there was no agreement in the promotion of two other sports, the 2012 programme featured just 26 sports.

---

**Question Answer Examples**

Q: How many nations participating in the olympics?
A: "207"

Q: When are The Olympic Games are normally held?
A: "Every four years"